In [14]:
import os
import pickle
import glob
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK

from nixtla.losses.numpy import mae, mape, smape, rmse, pinball_loss

In [27]:
feature = 'ART'
pooling = '1'
horizon = 30
experiment_id = '20210504'
filename = f'../results/result_{horizon}_{feature}_{pooling}_{experiment_id}.p'

In [28]:
results = pickle.load(open(filename,'rb'))

In [29]:
horizon_id = 0
horizon = results['horizon']
y_true = results['y_true']
y_hat = results['y_hat']

In [30]:
mae(y_true, y_hat)

20.498638

In [ ]:
    # plt.plot(horizons, mae_list)
    # plt.xlabel('Forecasting Horizon')
    # plt.ylabel('MAE')
    # plt.grid()
    # plt.savefig('horizon_vs_mae.pdf')